In [ ]:
# pip install azure-storage-blob

In [ ]:
import os
import sys
from dotenv import load_dotenv

from azure.storage.blob import BlobServiceClient, BlobClient, ContentSettings
from azure.core.exceptions import ResourceExistsError
# from azure.identity import DefaultAzureCredential
# from azure.identity import ClientSecretCredential
# from azure.identity import ManagedIdentityCredential

In [ ]:
# Read environment variables from a .env file
load_dotenv() # Load variables from .env file

In [ ]:
# Read work folders from environment variables
DOCUMENT_PRODUCED = os.environ.get("DOCUMENT_PRODUCED")
BLOB_CONNECTION_STRING = os.environ.get("BLOB_CONNECTION_STRING")
BLOB_CONTAINER_NAME = os.environ.get("BLOB_CONTAINER_NAME")

print(f"Produced folder: {DOCUMENT_PRODUCED}")
print(f"Blob connection string: {BLOB_CONNECTION_STRING}")
print(f"Blob container name: {BLOB_CONTAINER_NAME}")

In [ ]:
# Upload content of a folder to Azure Blob Storage

# Initialize Blob Service Client
try:
    blob_service_client = BlobServiceClient.from_connection_string(BLOB_CONNECTION_STRING)
    # blob_service_client = BlobServiceClient(account_url="https://<your_account_name>.blob.core.windows.net", credential=DefaultAzureCredential())
except Exception as e:
    print(f"Error creating BlobServiceClient: {e}")
    sys.exit(1)

# Create a container client
try:
    container_client = blob_service_client.get_container_client(BLOB_CONTAINER_NAME)
    # Ensure the container exists
    container_client.create_container()
except ResourceExistsError:
    print(f"Container '{BLOB_CONTAINER_NAME}' already exists.")
    # pass  # Ignore if already exists
except Exception as e:
    print(f"Error creating container: {e}")
    sys.exit(1)

# Upload files recursively
for root, dirs, files in os.walk(DOCUMENT_PRODUCED):
    for filename in files:
        file_path = os.path.join(root, filename)
        # Create a blob path relative to the DOCUMENT_PRODUCED
        blob_path = os.path.relpath(file_path, DOCUMENT_PRODUCED).replace("\\", "/")

        print(f"Uploading {file_path} to {blob_path}...")

        try:
            with open(file_path, "rb") as data:
                container_client.upload_blob(
                    name=blob_path,
                    data=data,
                    overwrite=True,
                    content_settings=ContentSettings(content_type="application/octet-stream")
                )
        except Exception as e:
            print(f"Error uploading {file_path}: {e}")

print("✅ Upload completed.")

# Upload files to the container
# for root, _, files in os.walk(local_folder_path):
#     for file in files:
#         file_path = os.path.join(root, file)
#         blob_client = container_client.get_blob_client(file)
#         try:
#             with open(file_path, "rb") as data:
#                 blob_client.upload_blob(data)
#             print(f"Uploaded {file} to {container_name}.")
#         except Exception as e:
#             print(f"Error uploading {file}: {e}")
